In [3]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset
import pdb
#### import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import Sequence
import pandas as pd
import os 
tf.compat.v1.enable_v2_behavior
# Import tensornetwork
import tensornetwork as tn
# Set the backend to tesorflow
# (default is numpy)
tn.set_default_backend("tensorflow")
batch_size = 32
learning_rate = 0.0001
class LIDC(Sequence):
    def __init__(self, rater=4, split='Train', data_dir = './', transform=None, num_classes=2):
        super().__init__()
        self. num_classes = num_classes
        self.data_dir = data_dir
        self.rater = rater
        self.transform = transform
        self.data, self.targets = torch.load(data_dir+split+'.pt')
        self.targets = self.targets.type(torch.FloatTensor)		   
    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        image, label = self.data[index], self.targets[index]
        if self.rater == 4:
            label = (label.sum() > 2).type_as(self.targets)
        else:
            label = label[self.rater]
        image = image.type(torch.FloatTensor)/255.0
        if self.transform is not None:
            image = self.transform(image)
        label = label.numpy().astype(np.int64)
        label = np.eye(self.num_classes)[label].astype(np.int64)
        return image.numpy().reshape((128, 128, 1)), label




class Batcher(Sequence):
    """Assemble a sequence of things into a sequence of batches."""
    def __init__(self, sequence, batch_size=16):
        self._batch_size = batch_size
        self._sequence = sequence
        self._idxs = np.arange(len(self._sequence))

    def __len__(self):
        return int(np.ceil(len(self._sequence) / self._batch_size))

    def __getitem__(self, i):
        if i >= len(self):
            raise IndexError("Index out of bounds")

        start = i*self._batch_size
        end = min(len(self._sequence), start+self._batch_size)
        data = [self._sequence[j] for j in self._idxs[start:end]]
        inputs = [d[0] for d in data]
        outputs = [d[1] for d in data]

        return self._stack(inputs), self._stack(outputs)

    def _stack(self, data):
        if data is None:
            return None

        if not isinstance(data[0], (list, tuple)):
            return np.stack(data)

        seq = type(data[0])
        K = len(data[0])
        data = seq(
            np.stack([d[k] for d in data])
            for k in range(K)
        )

        return data

    def on_epoch_end(self):
        np.random.shuffle(self._idxs)
        self._sequence.on_epoch_end()

In [2]:
data_path = '/datacommons/carin/fk43/LIDC/'
dataset_train = LIDC(split='Train', data_dir=data_path,
						transform=None,rater=4)
dataset_valid = LIDC(split='Valid', data_dir=data_path,
						transform=None,rater=4)
dataset_test = LIDC(split='Test', data_dir=data_path,
						transform=None,rater=4)



In [3]:
dataset_train.__getitem__(1)[0].shape

(128, 128, 1)

In [4]:
train_data = Batcher(dataset_train, batch_size=batch_size)
val_data = Batcher(dataset_valid, batch_size=batch_size)
test_data = Batcher(dataset_test, batch_size=batch_size)

In [5]:
class GridMERAin16(tf.keras.layers.Layer):
    
    def __init__(self, kernel_dims, bond_dims, output_dims):
        super(GridMERAin16, self).__init__()
        # Create the variables for the layer.
        # In this case, the input tensor is (, 1936), we factorize it into a tensor (, 11, 11, 16)
        # first_dim: output shape?
        # second_dim: connect with data tensor
        # third_dim: inter-connect
        in_dims = int((kernel_dims//4)**2)
        self.entanglers = []
        self.isometries= []
        self.kernel_dims = kernel_dims
        self.output_dims = output_dims
        #entanglers
        self.entanglers1 = tf.Variable(tf.random.normal
                                             (shape=(in_dims, in_dims, 
                                                     in_dims, in_dims, bond_dims, bond_dims, bond_dims, bond_dims),
                                              stddev=1/5000), 
                                              trainable=True)
        self.entanglers2 = tf.Variable(tf.random.normal
                                             (shape=(bond_dims, bond_dims, 
                                                     bond_dims, bond_dims, bond_dims, bond_dims, bond_dims, bond_dims),
                                              stddev=1/5000), 
                                              trainable=True)
        # isometries
        self.isometries1 = [tf.Variable(tf.random.normal(shape=(in_dims, in_dims, in_dims, 
                                                                            bond_dims, bond_dims)
                                                                     , stddev=1.0/10*5000),
                                            trainable=True), 
                           tf.Variable(tf.random.normal(shape=(in_dims, in_dims, bond_dims, 
                                                                            in_dims, bond_dims)
                                                                     , stddev=1.0/10*5000),
                                            trainable=True),
                           tf.Variable(tf.random.normal(shape=(in_dims, bond_dims, in_dims, 
                                                                            in_dims, bond_dims)
                                                                     , stddev=1.0/10*5000),
                                            trainable=True),
                           tf.Variable(tf.random.normal(shape=(bond_dims, in_dims, in_dims, 
                                                                            in_dims, bond_dims)
                                                                     , stddev=1.0/10*5000),
                                            trainable=True)]
        
        self.isometries2 = tf.Variable(tf.random.normal(shape=(bond_dims, bond_dims, bond_dims, 
                                                                            bond_dims, output_dims)
                                                                     , stddev=1.0/10*5000),
                                            trainable=True)

        #print(self.final_mps.shape)
        self.bias = tf.Variable(tf.zeros(shape=(output_dims,)), name="bias", trainable=True)


    def call(self, inputs):
        # Define the contraction.
        # We break it out so we can parallelize a batch using tf.vectorized_map.
        def f(input_vec, entanglers1, entanglers2, isometries1, isometries2, bias_var, kernel_dims):
            input_vv = []
            step = int(kernel_dims//4)
            for i in range(4):
                for ii in range(4):
                    input_vv.append(tf.reshape(input_vec[i*step:i*step+step, ii*step:ii*step+step, 0], (1, step**2)))
            input_vec = tf.concat(input_vv, axis=0)
            input_vec = tf.reshape(input_vec, (16, step**2))
            input_vec = tf.unstack(input_vec)
            input_nodes = []
            for e_iv in input_vec:
                input_nodes.append(tn.Node(e_iv))
            
            e_nodes1 = tn.Node(entanglers1)
            e_nodes2 = tn.Node(entanglers2)
                
                                     
            isometries_nodes1 = []
            for eiso in isometries1:
                isometries_nodes1.append(tn.Node(eiso))
            isometries_nodes2 = tn.Node(isometries2)
            
            
            e_nodes1[0] ^ input_nodes[5][0]
            e_nodes1[1] ^ input_nodes[6][0]
            e_nodes1[2] ^ input_nodes[9][0]
            e_nodes1[3] ^ input_nodes[10][0]

            e_nodes1[4] ^ isometries_nodes1[0][3]
            e_nodes1[5] ^ isometries_nodes1[1][2]
            e_nodes1[6] ^ isometries_nodes1[2][1]
            e_nodes1[7] ^ isometries_nodes1[3][0]     
            
            input_nodes[0][0] ^ isometries_nodes1[0][0]
            input_nodes[1][0] ^ isometries_nodes1[0][1]
            input_nodes[4][0] ^ isometries_nodes1[0][2]
            
            input_nodes[2][0] ^ isometries_nodes1[1][0]
            input_nodes[3][0] ^ isometries_nodes1[1][1]
            input_nodes[7][0] ^ isometries_nodes1[1][3]
            
            input_nodes[8][0] ^ isometries_nodes1[2][0]
            input_nodes[12][0] ^ isometries_nodes1[2][2]
            input_nodes[13][0] ^ isometries_nodes1[2][3]
            
            input_nodes[11][0] ^ isometries_nodes1[3][1]
            input_nodes[14][0] ^ isometries_nodes1[3][2]
            input_nodes[15][0] ^ isometries_nodes1[3][3]
            
            
            isometries_nodes1[0][4] ^ e_nodes2[0]
            isometries_nodes1[1][4] ^ e_nodes2[1]
            isometries_nodes1[2][4] ^ e_nodes2[2]
            isometries_nodes1[3][4] ^ e_nodes2[3]

            e_nodes2[4] ^ isometries_nodes2[0]
            e_nodes2[5] ^ isometries_nodes2[1]
            e_nodes2[6] ^ isometries_nodes2[2]
            e_nodes2[7] ^ isometries_nodes2[3]

                            
            nodes = tn.reachable(isometries_nodes2)
            result = tn.contractors.greedy(nodes)
            result = result.tensor
            #print(result)
            #result = (c @ b).tensor
            # Finally, add bias.
            return result + bias_var
            #return result

        # To deal with a batch of items, we can use the tf.vectorized_map function.
        # https://www.tensorflow.org/api_docs/python/tf/vectorized_map
        output = tf.vectorized_map(lambda vec: f(vec, self.entanglers1, self.entanglers2,
                                                 self.isometries1,  self.isometries2, self.bias, self.kernel_dims), inputs)
        return tf.reshape(output, (-1, self.output_dims))
    
from tensorflow.keras.layers import Lambda, Input, Concatenate, Reshape, Softmax, Dense, Flatten
from tensorflow.keras.models import Model, Sequential

def get_model(input_shape=(64, 64, 1)):
    x_in = Input(shape=input_shape)
    x_out_list = []
    for i in range(8):
        for j in range(8):
            subx = Lambda(lambda x:x[:, 8*i:8*(i+1),8*j:8*(j+1),:] )(x_in)
            x_out_list.append(GridMERAin16(kernel_dims=8, bond_dims=2, output_dims=1)(subx))
    x_out = Concatenate(axis=1)(x_out_list)
    x_out = Reshape(target_shape=(8, 8, 1))(x_out)
    y = GridMERAin16(kernel_dims=8, bond_dims=2, output_dims=2)(x_out)
    y = Softmax()(y)
    return Model(inputs=x_in, outputs=y)

def get_dense_model(input_shape=(64, 64, 1)):
    x_in = Input(shape=input_shape)
    x_out_list = []
    for i in range(8):
        for j in range(8):
            subx = Lambda(lambda x:x[:, 16*i:16*(i+1),16*j:16*(j+1),:] )(x_in)
            x_out_list.append(GridMERAin16(kernel_dims=16, bond_dims=2, output_dims=2)(subx))
    x_out = Concatenate(axis=1)(x_out_list)
    x_out = Flatten()(x_out)
    y = Dense(2, activation='softmax')(x_out)
    #x_out = Reshape(target_shape=(8, 8, 1))(x_out)
    #y = GridMERAin16(kernel_dims=8, bond_dims=2, output_dims=2)(x_out)
    #y = Softmax()(y)
    return Model(inputs=x_in, outputs=y)

def get_loop_model(input_shape=(64, 64, 1)):
    x_in = Input(shape=input_shape)
    x_out_list = []
    MERA_layer = GridMERAin16(kernel_dims=16, bond_dims=2, output_dims=4)
    for i in range(4):
        for j in range(4):
            subx = Lambda(lambda x:x[:, 16*i:16*(i+1),16*j:16*(j+1),:] )(x_in)
            x_out_list.append(MERA_layer(subx))
    x_out = Concatenate(axis=1)(x_out_list)
    x_out = Flatten()(x_out)
    y = Dense(2, activation='softmax')(x_out)
    #x_out = Reshape(target_shape=(8, 8, 1))(x_out)
    #y = GridMERAin16(kernel_dims=8, bond_dims=2, output_dims=2)(x_out)
    #y = Softmax()(y)
    return Model(inputs=x_in, outputs=y)

def get_deep_model(input_shape=(64, 64, 1)):
    x_in = Input(shape=input_shape)
    x_in2_list = []
    for i in range(16):
        for j in range(16):
            subx = Lambda(lambda x:x[:, 4*i:4*(i+1),4*j:4*(j+1),:] )(x_in)
            x_in2_list.append(GridMERAin16(kernel_dims=1, bond_dims=2, output_dims=1)(subx))
    x_in2 = Concatenate(axis=1)(x_in2_list)
    x_in2 = Reshape(target_shape=(16, 16, 1))(x_in2)
    x_out_list = []
    for i in range(4):
        for j in range(4):
            subx = Lambda(lambda x:x[:, 4*i:4*(i+1),4*j:4*(j+1),:] )(x_in2)
            x_out_list.append(GridMERAin16(kernel_dims=1, bond_dims=2, output_dims=1)(subx))
    x_out = Concatenate(axis=1)(x_out_list)
    x_out = Reshape(target_shape=(4, 4, 1))(x_out)
    y = GridMERAin16(kernel_dims=1, bond_dims=2, output_dims=2)(x_out)
    y = Softmax()(y)
    return Model(inputs=x_in, outputs=y)

In [6]:
def get_loop_model(input_shape=(128, 128, 1)):
    x_in = Input(shape=input_shape)
    x_out_list = []
    MERA_layer = GridMERAin16(kernel_dims=4, bond_dims=2, output_dims=2)
    for i in range(32):
        for j in range(32):
            subx = Lambda(lambda x:x[:, 4*i:4*(i+1),4*j:4*(j+1),:] )(x_in)
            x_out_list.append(MERA_layer(subx))
    x_out = Concatenate(axis=1)(x_out_list)
    x_out = Flatten()(x_out)
    y = Dense(2, activation='softmax')(x_out)
    #x_out = Reshape(target_shape=(8, 8, 1))(x_out)
    #y = GridMERAin16(kernel_dims=8, bond_dims=2, output_dims=2)(x_out)
    #y = Softmax()(y)
    return Model(inputs=x_in, outputs=y)

def get_dense_model(input_shape=(128, 128, 1)):
    x_in = Input(shape=input_shape)
    x_out_list = []
    for i in range(32):
        for j in range(32):
            subx = Lambda(lambda x:x[:, 4*i:4*(i+1),4*j:4*(j+1),:] )(x_in)
            x_out_list.append(GridMERAin16(kernel_dims=4, bond_dims=2, output_dims=2)(subx))
    x_out = Concatenate(axis=1)(x_out_list)
    x_out = Flatten()(x_out)
    y = Dense(2, activation='softmax')(x_out)
    #x_out = Reshape(target_shape=(8, 8, 1))(x_out)
    #y = GridMERAin16(kernel_dims=8, bond_dims=2, output_dims=2)(x_out)
    #y = Softmax()(y)
    return Model(inputs=x_in, outputs=y)

In [ ]:
MERA_model_64 = get_loop_model(input_shape=(128, 128, 1))

MERA_model_64.summary()

In [ ]:
MERA_model_64.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate, clipnorm=0.2), metrics=['accuracy'])
MERA_model_64_hist = MERA_model_64.fit_generator(train_data, validation_data=val_data, epochs=50, verbose=1)

In [ ]:
MERA_model_64.evaluate_generator(test_data, int(np.ceil(1000/batch_size)), workers = 1)

tn_loss = MERA_model_64_hist.history['loss']
tn_acc = MERA_model_64_hist.history['accuracy']


np.savetxt('loss_MERA64.out', tn_loss, delimiter=',')  
np.savetxt('acc_MERA64.out', tn_acc, delimiter=',') 